### 기본 라이브러리 불러오기

In [1]:
# 라이브러리 임포트
import os
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

---
### 자치구_동 데이터 가져오기

In [4]:
start_time = time.time()
df_parquet_dong_gu = pd.read_parquet('G:\\내 드라이브\\DataSet\\_최종 병합 파일\\' + '시도_시군구_읍면동' + '.parquet')
print("read_parquet time :", time.time() - start_time)

read_parquet time : 0.08543801307678223


In [6]:
df_parquet_dong_gu.head()

,시도,시군구,읍면동,시도명,시군구명,읍면동명
0,11000,11010,1101053,서울특별시,종로구,사직동
1,11000,11010,1101054,서울특별시,종로구,삼청동
2,11000,11010,1101055,서울특별시,종로구,부암동
3,11000,11010,1101056,서울특별시,종로구,평창동
4,11000,11010,1101057,서울특별시,종로구,무악동


In [8]:
df_sum = pd.DataFrame()

In [10]:
# _years = ['2020', '2021', '2022', '2023', '2024']
_years = ['2022']

In [12]:
path_dateset = 'G:\\내 드라이브\\DataSet\\'
path_sub = '_최종 병합 파일\\서울 생활이동\\'

---
### 생활이동_행정동 데이터 가공 (2020 ~ 2024)

- 나이 : 20 ~ 60
- 시간
    - 출근(7 ~ 8) : 실제 시작 (7 ~ 9)
    - 퇴근(17 ~ 18) : 실제 시작 (17 ~ 19)
- 이동유형
    - 출근('HW', 'EW')
        - 'HW' :
        - 'EW' : 
    - 퇴근('WH', 'EH')
        - 'WH' :
        - 'EH' : 

In [ ]:
for cnt, _year in enumerate(_years):
    
    path = path_dateset + path_sub + _year + '\\'    
    savefile = '생활이동_행정동_출퇴근_' + _year   # Ex) 생활이동_행정동_출퇴근_2024
    print(f'path : {path}, savefile : {savefile}')
    
    # 파일 경로명 변경
    file_list = os.listdir(path)
    file_lists = [file for file in file_list if file.endswith((".csv", ".xlsx"))]
    file_lists.sort()
    print('file_lists : \n', '\n '.join(file_lists))
    print('===========================================')

    #try:
    for _, filename in enumerate(file_lists):        
        print('filename : ', filename + ', encoding : ', em.Get_ExcelEncoding(path + filename))
        start_time_full = time.time()

        # 서울 생활이동 데이터 불러오기
        start_time = time.time()
        print('path + filename : ', path + filename)

        dtype={'이동인구(합)': 'object', '평균 이동 시간(분)': 'float64'}

        # 2024.11.04 duzin
        # dask = er.Read_dask(path + filename, dtype)
        dask = er.Read_Chunk(path + filename)
        print("Read_dask(" + filename + ") time :", time.time() - start_time)

        df_dask_rtn = dask
        del dask  # 메모리 정리
        # from dask.diagnostics import ProgressBar
        # from time import sleep
        # pbar = ProgressBar()
        # pbar.register()
        # start_time = time.time()
        # df_dask_rtn = dask.compute()
        # del dask  # 메모리 정리
        # print("compute time(" + filename + ") :", time.time() - start_time)

        # 필요한 조건 설정
        start_time = time.time()
        cond1 = df_dask_rtn['요일'].isin(['월', '화', '수', '목', '금'])
        cond2 = df_dask_rtn['나이'].isin([20, 25, 30, 35, 40, 45, 50, 55, 60])   # 20 ~ 60
        cond3 = (df_dask_rtn['이동유형'].isin(['HW', 'EW']) | df_dask_rtn['이동유형'].isin(['WH', 'EH']))
        cond4 = (df_dask_rtn['도착시간'].isin([7, 8]) | df_dask_rtn['도착시간'].isin([17, 18]))
        dask = df_dask_rtn[cond1 & cond2 & cond3 & cond4]
        print("search Condition(" + filename + ") :", time.time() - start_time)
        
        # '이동인구(합)' 컬럼 float형으로 변경
        df_dask_rtn.loc[df_dask_rtn['이동인구(합)'] == '*','이동인구(합)'] = '0'
        df_dask_rtn = df_dask_rtn.astype({ '이동인구(합)' : 'float' })

        # 출퇴근 컬럼 추가
        df_dask_rtn.loc[(df_dask_rtn['이동유형'].isin(['HW', 'EW']))&(df_dask_rtn['도착시간'].isin([7, 8])),'출퇴근'] = '출근'
        df_dask_rtn.loc[(df_dask_rtn['이동유형'].isin(['WH', 'EH']))&(df_dask_rtn['도착시간'].isin([17, 18])),'출퇴근'] = '퇴근'
        df_dask_rtn = df_dask_rtn[~df_dask_rtn['출퇴근'].isnull()]

        # '대상연월', '도착 행정동 코드', '출퇴근' 기준으로, '이동인구(합)'의 계 구함
        start_time = time.time()
        df_dask_rtn = df_dask_rtn.groupby(['대상연월', '출발 행정동 코드', '도착 행정동 코드', '출퇴근'])['이동인구(합)'].sum().reset_index()
        print("groupby time :", time.time() - start_time)

        df_dask_rtn.rename(columns = {"도착 행정동 코드": "읍면동"}, inplace = True)
        df_dask_rtn = pd.merge(left = df_dask_rtn , right = df_parquet_dong_gu, how = "left", on = "읍면동")

        print("=============================================================")
        print('df_dask_rtn shape : ', df_dask_rtn.shape)
        print("Read full time :", time.time() - start_time_full)

        df_sum = pd.concat([df_dask_rtn, df_sum], axis = 0)
        del df_dask_rtn  # 메모리 정리

    # for문 종료 후, 실행
    df_sum.to_parquet(path + savefile + '.parquet', index=False)
    del df_sum
    
    # except:
    #     print('- Err : ', savefile)  
    

path : G:\내 드라이브\DataSet\_최종 병합 파일\서울 생활이동\2022\, savefile : 생활이동_행정동_출퇴근_2022
file_lists : 
 생활이동_행정동_2022.01.csv
 생활이동_행정동_2022.02.csv
 생활이동_행정동_2022.03.csv
 생활이동_행정동_2022.04.csv
 생활이동_행정동_2022.05.csv
 생활이동_행정동_2022.06.csv
 생활이동_행정동_2022.07.csv
 생활이동_행정동_2022.08.csv
 생활이동_행정동_2022.09.csv
 생활이동_행정동_2022.10.csv
 생활이동_행정동_2022.11.csv
 생활이동_행정동_2022.12.csv
filename :  생활이동_행정동_2022.01.csv, encoding :  utf-8
path + filename :  G:\내 드라이브\DataSet\_최종 병합 파일\서울 생활이동\2022\생활이동_행정동_2022.01.csv
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Processed a chunk of size: 1000000
 Proce